In [1]:
import glob
import re
import os
import csv
from pprint import pprint
from tqdm import *

In [2]:
path = "/mnt/sdb1/BuildServer"
readme_path = path + "/BuildServer.ReadMe.txt"
traces_path = path + "/Traces"
#output_oath = path + "/Combined"
output_path = "/home/tiratatp/Repositories/snia_traces/T7/BuildServer/Combined"

In [3]:
# read from readme
traces = []
with open(readme_path) as readme:
    for i, line in enumerate(readme):
        if i < 58:
            continue
        cols = re.findall("(.*?)\: (.*?)(?:\,|$)", line)
        if len(cols) == 0:
            continue
        traces.append(cols)

# sort the filename based on start timestamp            
traces = sorted(traces, key=lambda x: int(x[4][1]))   
pprint(traces)

[[('OS Version', '6.0.6001'),
  (' Trace Size', '10112KB'),
  (' Events Lost', '0'),
  (' Buffers lost', '0'),
  (' Trace Start', '128407802885377025'),
  (' Trace Length', '15 min'),
  (' PointerSize', '8'),
  (' Trace Name', '24.hour.BuildServer.11-28-2007.07-24-PM.trace.etl\r')],
 [('OS Version', '6.0.6001'),
  (' Trace Size', '33920KB'),
  (' Events Lost', '0'),
  (' Buffers lost', '0'),
  (' Trace Start', '128407811968691251'),
  (' Trace Length', '15 min'),
  (' PointerSize', '8'),
  (' Trace Name', '24.hour.BuildServer.11-28-2007.07-39-PM.trace.etl\r')],
 [('OS Version', '6.0.6001'),
  (' Trace Size', '1235328KB'),
  (' Events Lost', '0'),
  (' Buffers lost', '0'),
  (' Trace Start', '128407821042489416'),
  (' Trace Length', '15 min'),
  (' PointerSize', '8'),
  (' Trace Name', '24.hour.BuildServer.11-28-2007.07-55-PM.trace.etl\r')],
 [('OS Version', '6.0.6001'),
  (' Trace Size', '1289344KB'),
  (' Events Lost', '0'),
  (' Buffers lost', '0'),
  (' Trace Start', '1284078301203

In [4]:
outputs = {}
try:
    os.mkdir(output_path)
except OSError:
    pass

# in microsecond
first_start_offset = int(traces[0][4][1])/10
first_io = None
last_io = None

for t in traces:    
    # in microsecond
    current_start_offset = int(t[4][1])/10
    
    # remove two last extensions
    filename = glob.glob("%s/%s*" % (traces_path, ".".join(t[7][1].split(".")[:-2])))
    print "Looking for:", ".".join(t[7][1].split(".")[:-2])
    filename = filename[0]
    print "Found:", filename
     
    with open(filename) as f:
        # skip header
        for line in f:
            if line[:9] == "EndHeader":
                break
        
        reader = csv.reader(f, delimiter=",", skipinitialspace=True)
        for tok in tqdm(reader):
            try:
                if tok[0] == "DiskWrite":
                    flags = 0
                elif tok[0] == "DiskRead":
                    flags = 1
                else:
                    continue
            except IndexError:
                continue
                
            t = {
                "time": (float(tok[1]) + (current_start_offset - first_start_offset)) / 1000.0,
                "devno": int(tok[8]),
                "blkno": int(tok[5], 16) / 512,
                "bcount": int(tok[6], 16) / 512,
                "flags": flags,
            };
            
            if t['devno'] not in outputs:
                outputs[t['devno']] = open("%s/disk%d.txt" % (output_path, t['devno']), "w")

            outputs[t['devno']].write("%s %d %d %d %d\n" % ("{0:.3f}".format(t['time']), t['devno'], t['blkno'], t['bcount'], t['flags']))
            #print "%s %d %d %d %d\n" % ("{0:.3f}".format(t['time']), t['devno'], t['blkno'], t['bcount'], t['flags'])
            #break
            
            if first_io is None:
                first_io = t
            last_io = t
    #break            

Looking for: 24.hour.BuildServer.11-28-2007.07-24-PM
Found: /mnt/sdb1/BuildServer/Traces/24.hour.BuildServer.11-28-2007.07-24-PM.trace.csv
Looking for: 24.hour.BuildServer.11-28-2007.07-39-PM
Found: /mnt/sdb1/BuildServer/Traces/24.hour.BuildServer.11-28-2007.07-39-PM.trace.csv
Looking for: 24.hour.BuildServer.11-28-2007.07-55-PM
Found: /mnt/sdb1/BuildServer/Traces/24.hour.BuildServer.11-28-2007.07-55-PM.trace.csv
Looking for: 24.hour.BuildServer.11-28-2007.08-10-PM
Found: /mnt/sdb1/BuildServer/Traces/24.hour.BuildServer.11-28-2007.08-10-PM.trace.csv
Looking for: 24.hour.BuildServer.11-28-2007.08-25-PM
Found: /mnt/sdb1/BuildServer/Traces/24.hour.BuildServer.11-28-2007.08-25-PM.trace.csv
Looking for: 24.hour.BuildServer.11-28-2007.08-40-PM
Found: /mnt/sdb1/BuildServer/Traces/24.hour.BuildServer.11-28-2007.08-40-PM.trace.csv
Looking for: 24.hour.BuildServer.11-28-2007.08-55-PM
Found: /mnt/sdb1/BuildServer/Traces/24.hour.BuildServer.11-28-2007.08-55-PM.trace.csv
Looking for: 24.hour.BuildS

In [5]:
for f in outputs:
    outputs[f].close()

In [6]:
print "The trace is", last_io['time'] - first_io['time'], "ms or about", (last_io['time'] - first_io['time'])/3600000, "hours"

The trace is 87217993.476 ms or about 24.22722041 hours


In [7]:
print "From the readme, the trace is ", (int(traces[-1][4][1]) - int(traces[0][4][1]))/36000000000. , "hours"

From the readme, the trace is  23.9762399309 hours
